# Eksempel av MLFlow Tracking

In [1]:
import mlflow
import pandas as pd
from mlflow.models import infer_signature
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split


# Modelltrening
Man trener en modell som vanlig, her har vi hentet inn iris-datasettet fra sklearn.datasets og lager en enkel LogisticRegression-modell.
 

In [ ]:

# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)


# Vi kan logge resultatene våre i MLflow


In [ ]:

# Opprett et nytt MLflow-eksperiment
mlflow.set_experiment("Iris")

# Start en MLflow-run
with mlflow.start_run():
    # Logg hyperparametrene
    mlflow.log_params(params)

    # Logg tapsmetrikken
    mlflow.log_metric("accuracy", accuracy)

    # Sett en tag som vi kan bruke for å minne oss selv på hva denne kjøringen var for
    mlflow.set_tag("Opplæringsinfo", "Grunnleggende LR-modell for irisdata")

    # Utled modellens signatur
    signature = infer_signature(X_train, lr.predict(X_train))

    # Logg modellen
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="iris",
    )


# Vi har lagret modellen, hva nå?
Etter at modellen er lagret så kan vi studere resultatene i MLflow UI.
For å gjøre dette kan vi kjøre følgende kommando i terminalen:
```bash
    mlflow ui
```

Dette vil starte en lokal server som vi kan åpne i nettleseren vår.


# Gjenbruk
Vi har studert modellene og våre og funnet den vi vil ha, hvordan gjenbruker vi den? 

In [ ]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]